In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Dropout
from keras import metrics



%matplotlib inline

In [29]:
# Load the dataset
df = pd.read_excel(r'/FRAUD/ccfraudTrain.xlsx')

df1 = pd.read_excel(r'/FRAUD/ccfraudTest.xlsx')
# For snakey diagram
#df1 = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\Sankey_data.xlsx')

In [30]:
#print (df['trans_date_trans_time'].head())
from datetime import datetime as dt
import pytz  # Import the pytz module for time zones

df['trans_date_trans_time_HR'] = df['trans_date_trans_time'].dt.strftime("%H")

print (df.head())

df1['trans_date_trans_time_HR'] = df1['trans_date_trans_time'].dt.strftime("%H")

print (df1.head())

df['trans_date_trans_time_HR'] = df['trans_date_trans_time_HR'].astype(int)

df1['trans_date_trans_time_HR'] = df1['trans_date_trans_time_HR'].astype(int)

df['trans_day_of_week'] = df['trans_date_trans_time'].dt.dayofweek

df1['trans_day_of_week'] = df1['trans_date_trans_time'].dt.dayofweek

   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652090   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street  ...      long city_pop  \
0    Banks      F                561 Perry Cove  ...  -81.1781     3495   
1     Gill      F  43039 Riley Greens Suite 393  ... -11

In [31]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['trans_date'] = df['trans_date_trans_time'].dt.date

df1['trans_date_trans_time'] = pd.to_datetime(df1['trans_date_trans_time'])
df1['trans_date'] = df1['trans_date_trans_time'].dt.date

In [32]:
df = df.drop('unix_time', axis=1)
df1 = df1.drop('unix_time', axis=1)

df = df.drop('first', axis=1)
df1 = df1.drop('first', axis=1)

df = df.drop('last', axis=1)
df1 = df1.drop('last', axis=1)

df = df.drop('street', axis=1)
df1 = df1.drop('street', axis=1)

df = df.drop('trans_num', axis=1)
df1 = df1.drop('trans_num', axis=1)

df = df.drop('trans_date_trans_time', axis=1)
df1 = df1.drop('trans_date_trans_time', axis=1)


df = df.drop('trans_date', axis=1)
df1 = df1.drop('trans_date', axis=1)


df = df.drop('dob', axis=1)
df1 = df1.drop('dob', axis=1)




In [7]:
# Scale the Numeric column
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['amt'] = scaler.fit_transform(df['amt'].values.reshape(-1, 1))
df['cc_num'] = scaler.fit_transform(df['cc_num'].values.reshape(-1, 1))
df['zip'] = scaler.fit_transform(df['zip'].values.reshape(-1, 1))
df['lat'] = scaler.fit_transform(df['lat'].values.reshape(-1, 1))
df['long'] = scaler.fit_transform(df['long'].values.reshape(-1, 1))
df['city_pop'] = scaler.fit_transform(df['city_pop'].values.reshape(-1, 1))
df['merch_lat'] = scaler.fit_transform(df['merch_lat'].values.reshape(-1, 1))
df['merch_long'] = scaler.fit_transform(df['merch_long'].values.reshape(-1, 1))

In [33]:
# Label encode the Categorical column
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['gender'] = encoder.fit_transform(df['gender'])
df['merchant'] = encoder.fit_transform(df['merchant'])
df['category'] = encoder.fit_transform(df['category'])
df['city'] = encoder.fit_transform(df['city'])
df['state'] = encoder.fit_transform(df['state'])
df['job'] = encoder.fit_transform(df['job'])

# Extract features from 'trans_date_trans_time' Like did above

In [34]:
# Preprocess your data
# This is a placeholder. You need to fill in with your own preprocessing steps.
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

In [35]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
y_train = tf.cast(y_train, tf.float32)


y_test = y_test.astype('float32')

In [37]:
# Create the Feedforward Neural Network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification, so 'sigmoid' activation
])

In [38]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = [metrics.F1Score()])

In [39]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
26215/26215 [==============================] - 59s 2ms/step - loss: 60950275162112.0000 - f1_score: 0.0113 - val_loss: 0.0348 - val_f1_score: 0.0112
Epoch 2/10
 1559/26215 [>.............................] - ETA: 40s - loss: 0.0362 - f1_score: 0.0118

KeyboardInterrupt: ignored

In [21]:
# Evaluate the F1 score
f1_score = model.evaluate(X_test, y_test, verbose=0)[1]

# Print the F1 score
print(f'F1 Score: {str(f1_score*100)[:6]}%')

F1 Score: [1.120%


In [22]:
# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                1152      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3265 (12.75 KB)
Trainable params: 3265 (12.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import metrics

# Define your model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # Adjust input shape
    Dropout(0.5),  # Add dropout for regularization
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'sigmoid' activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[metrics.F1Score()])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, f1_score = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, F1 Score: {f1_score}')

# Plot the training history to check for overfitting
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Epoch 1/10
26215/26215 [==============================] - 66s 2ms/step - loss: 40.1377 - f1_score: 0.0116 - val_loss: 0.0365 - val_f1_score: 0.0112
Epoch 2/10
26215/26215 [==============================] - 58s 2ms/step - loss: 0.0694 - f1_score: 0.0115 - val_loss: 0.0350 - val_f1_score: 0.0112
Epoch 3/10
26215/26215 [==============================] - 58s 2ms/step - loss: 0.0854 - f1_score: 0.0114 - val_loss: 0.0356 - val_f1_score: 0.0112
Epoch 4/10
26215/26215 [==============================] - 56s 2ms/step - loss: 0.0405 - f1_score: 0.0114 - val_loss: 0.0348 - val_f1_score: 0.0112
Epoch 5/10
26206/26215 [============================>.] - ETA: 0s - loss: 0.0428 - f1_score: 0.0115

KeyboardInterrupt: ignored